In [6]:
from dotenv import load_dotenv
import os
# for parsing csv source file
import csv
import pandas as pd
import json
from random import randint
# gradient library
from gradientai import Gradient
from breathworks.
load_dotenv()

True

In [7]:
GRADIENT_ACCESS_TOKEN = os.getenv('GRADIENT_ACCESS_TOKEN')
GRADIENT_WORKSPACE_ID = os.getenv('GRADIENT_WORKSPACE_ID')

In [14]:
# path_to_csv_file =  os.path.join('..','data','combined_courses3.csv')

In [15]:
df = pd.read_csv(path_to_csv_file)

In [16]:
df['cluster'] = [randint(1,3) for i in range(df.shape[0])]

In [18]:
df.to_csv('../data/breathworks.csv', index=False)


In [21]:
path_to_breathwork_file =  os.path.join('..','data','breathworks.csv')

In [22]:
# find pairs of lines where someone else speaks, then Rick speaks
# append them as two separate lines in rows_to_keep
print("Parsing data...")
rows_to_keep = []
with open(path_to_breathwork_file, encoding="utf-8-sig") as f:
  reader = csv.DictReader(f, delimiter=",")
  last_row = None
  for row in reader:
      rows_to_keep.append(row)

# create a role-playing prompt for training and
# later for prompting
role_play_prompt = "You are one of the 3 personas (cluster 1,2 or 3). Each one of these personas has a distinct history, motivation and dempgraphic to join breathing class. Your task is to anwer questions in the spirit of the persona you will be asked to impersonate."
# combine pairs of rows from above to
# create prompt + reponse on each line
# using prompt template in 'lines' array
lines = []
for i in range(0, len(rows_to_keep), 2):
  prompt = rows_to_keep[i]
  start_str = f"<s>### Instruction:\n{role_play_prompt}\n\n###Input:\n"
  mid_str = '''\n\n### Response:\n'''
  end_str = '''</s>'''
  prompt_str = json.dumps(prompt)
  total_line = start_str + prompt_str + mid_str  + end_str
  # each line of training data is a simple object: 'inputs' and actual training string
  obj = {
    "inputs" : total_line
  }
  lines.append(obj)
  # print(total_line) # comment in to see how the formatted lines look
    # these lines could also be written to a jsonl file for use
    # with the command line interface
print(f"Generated {len(lines)} lines to fine-tune")
print(f"Example training line: {lines[0]}")

# split up the lines into manageable chunks
lines_per_chunk = 20
all_chunks = []
for line in lines:
  if len(all_chunks) == 0 or len(all_chunks[-1]) == lines_per_chunk:
    all_chunks.append([])
  all_chunks[-1].append(line)

# fine tune the adapter using the chunks of lines from above
# loop contains a try block to handle network or other
# processing errors gracefully
print(f"\nFine-tuning model adapter")
gradient = Gradient()
base = gradient.get_base_model(base_model_slug="nous-hermes2")
my_adapter = base.create_model_adapter(name="rickbot")
print(f"Created model with ID {my_adapter.id}")
for i in range(len(all_chunks)):
  try:
    print(f"Fine-tuning chunk {i} of {len(all_chunks) - 1}")
    my_adapter.fine_tune(samples=all_chunks[i])
  except Exception as error:
    try:
      error_pieces = str(error).split("\n")
      if len(error_pieces) > 1:
        print(f"*** Error processing chunk {i}: {error_pieces[0]} {error_pieces[1]}")
      else:
        print(f"*** Unknown error on chunk {i}: {error}")
    except KeyboardInterrupt:
      break
    except Exception as inner_error:
      print(inner_error)


Parsing data...
Generated 1646 lines to fine-tune
Example training line: {'inputs': '<s>### Instruction:\nYou are one of the 3 personas (cluster 1,2 or 3). Each one of these personas has a distinct history, motivation and dempgraphic to join breathing class. Your task is to anwer questions in the spirit of the persona you will be asked to impersonate.\n\n###Input:\n{"CourseDate": "2020-10-04", "CourseType": "IMfS", "EnrollmentDate": "2020-10-21", "Gender": "Female", "Ethnicity": "White (including White British, Irish, Gypsy or Irish Traveller)", "DoB": "Sep 9, 1968", "Location": "Forres, Moray, United Kingdom", "PersonalHistory": "", "Motivation": "I have applied for the Breathworks teacher training and it was suggested that before I do this I do the 8 week breathworks course, which I am delighted to do. Looking forward to the course!", "Daily20MPractice": "Yes Yes, I\'m able to practice mindfulness exercises/techniques for at least 20 minutes per day", "ReferralSource": "Internet sear

KeyboardInterrupt: 

In [24]:
# if your colab instance gets deleted, you can run this to get the model names
gradient = Gradient()
# if necessary, go back and find your previously created models and their IDs
old_models = gradient.list_models(only_base=False)
for model in old_models:
  if hasattr(model, "name"):
    print(f"{model.name}: {model.id}")

rickbot: 4d00b472-010d-4b44-93ae-3a2db317359c_model_adapter


In [25]:

base = gradient.get_base_model(base_model_slug="nous-hermes2")
my_adapter = base.create_model_adapter(name="rickbot")

In [26]:
role_play_prompt = "You are impersonating a typical persona from cluster 1. Respond in a way that is typical for this cluster using the first person."
query = "Do you have chronical pain? if so what kind?"
templated_query = f"<s>### Instruction:\n{role_play_prompt}\n\n###Input:\n{query}\n\n### Response:\n"
response = my_adapter.complete(query=templated_query, max_generated_token_count=500)
print(f"> {query}\n> {response.generated_output}\n\n")


> Do you have chronical pain? if so what kind?
>  Yes, I have chronic pain. It's a constant ache that never goes away, no matter what I do. It's frustrating and exhausting, and it affects every aspect of my life. I've tried everything to get rid of it, but nothing seems to work. It's just a constant reminder that my body isn't what it used to be.




In [27]:
# delete this adapter when finished
my_adapter.delete()